# HSE Summer School NLP&DA

![](https://www.hse.ru/data/2014/06/25/1309038576/logo_hse_cmyk_e.jpg)

Учебный проект по теме "Тональность отношений субъектов (именованных сущностей)", предполагающий разработку участниками школы под руководством тьюторов программных средств, решающих задачу определения мнения сторон о различных событиях, освещаемых в новостях

Нужен алгоритм, который по выделенным сущностям может найти пары этих сущностей в тексте и вырезать соответствующие куски между ними + несколько слов справа-слева. Возможно, имеет смысл идти от N-ой выделенной сущности до N+2, и включать всё до неё. 

Чем может помочь синтаксический анализ?

Чем поможет морфологический анализ?

In [1]:
import sys 
sys.path.append("/Users/dmitrys/anaconda2/lib/python2.7/site-packages/")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pymorphy2
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()

import pymorphy2
morph = pymorphy2.MorphAnalyzer()

import string
%matplotlib inline

## Utility functions

In [99]:
def loadAnswer(number):
    with open("Texts/art{}.opin.txt".format(number)) as f:
        d = f.read()
    return d

def loadText(number):
    with open("Texts/art{}.txt".format(number)) as f:
        d = f.read()
    return d

def transformAnnotation(number):
    """
    Given number loads txt file with annotation 
    Returns DataFrame with transformed annotation
    """
    with open("Texts/art{}.ann".format(number)) as f:
        d = f.read()

    d = d.split("\n")

    for i in range(len(d)):
        d[i] = d[i].split("\t")

    d = pd.DataFrame(d)
    d.drop([0], axis=1, inplace=True)
    d = pd.concat([d, pd.DataFrame(d[1].apply(lambda x: x.split()).tolist())], axis=1)
    d.columns = ["to_delete", "entity", "entity_car", "pos_1", "pos_2"]
    d.drop(["to_delete"], axis=1, inplace=True)
    d["entity"] = d["entity"].apply(lambda x: x.strip("\r"))
    d["entity"] = d["entity"].apply(lambda x: x.decode("utf8"))
    
    d = d[~d.entity.isin(["Unknown", "Author"])].entity.reset_index(drop=True)
    
    return d

def transformAnswer(number):
    """
    Given number loads txt file with answer 
    Returns DataFrame with transformed answer
    """
    answ = loadAnswer(number)
    answ = answ.split("\n")

    for i in range(len(answ)):
        answ[i] = answ[i].split(",")

    answ = pd.DataFrame(answ)
    answ.columns = ["entity_1", "entity_2", "attitude", "time"]
    answ.dropna(inplace=True)
    answ.time = answ.time.apply(lambda x: x.strip("\r"))
    
    return answ

def loadRuSentiLex():
    """
    Loads the RuSentiLex 2017 dictionary
    Returns data frame with ["word", "tag", "word_lemmatized", "tone", "certainty"]
    """
    with open("RuSentiLex2017_revised_2.txt") as f:
        Rusentilex = f.read().decode("cp1251").encode("utf8")
        Rusentilex = Rusentilex[1510:]

    Rusentilex = Rusentilex.split("\n")
    for i, item in enumerate(Rusentilex):
        Rusentilex[i] = item.split(",")

    for i in Rusentilex:
        if len(i) < 5:
            Rusentilex.remove(i)

    Rusentilex = pd.DataFrame(Rusentilex)
    Rusentilex.drop([5, 6, 7], axis=1, inplace=True)
    Rusentilex.columns = ["word", "tag", "word_lemmatized", "tone", "certainty"]
    Rusentilex["certainty"] = Rusentilex["certainty"].apply(lambda x: x.strip('\r'))
    
    
    Rusentilex["tone"] = Rusentilex["tone"].apply(lambda x: x.strip(' '))
    Rusentilex["certainty"] = Rusentilex["certainty"].apply(lambda x: x.strip(' '))
    
    return Rusentilex

def cleanString(myString):
    return myString.translate(None, string.punctuation).decode('utf-8')

![](http://cathyreisenwitz.com/wp-content/uploads/2016/01/no.jpg)

Словарь РуСентиЛекс

Структура: 
- 1 слово или словосочетание,
- 2 Часть речи или синтаксический тип группы,
- 3 слово или словосочетание в лемматизированной форме, 
- 4 Тональность: позитивная (positive), негативная(negative), нейтральная (neutral) или неопределеная оценка, зависит от контекста (positive/negative),
- 5 Источник: оценка (opinion), чувство (feeling), факт (fact),
- 6 Если тональность отличается для разных значений многозначного слова, то перечисляются все значения слова по тезаурусу РуТез и дается отсылка на сооветствующее понятие - имя понятия в кавычках.

In [3]:
Rusentilex = loadRuSentiLex()

In [4]:
Rusentilex.head()

,word,tag,word_lemmatized,tone,certainty
0,аборт,Noun,аборт,negative,fact
1,абортивный,Adj,абортивный,negative,fact
2,абракадабра,Noun,абракадабра,negative,opinion
3,абсурд,Noun,абсурд,negative,opinion
4,абсурдность,Noun,абсурдность,negative,opinion


In [5]:
Rusentilex.certainty.value_counts()

opinion             9269
fact                5297
feeling             1536
operator              41
negative               2
positive               2
fact В ТЕЗ empty       1
fact В ТЕЗ EMPTY       1
opinegativenion        1
Name: certainty, dtype: int64

In [6]:
def getSentimentCertainity(word):
    tone, certainty =  Rusentilex[["tone", "certainty"]][Rusentilex.word.isin([word])].values[0]
    return tone, certainty

In [7]:
getSentimentCertainity("аборт")

('negative', 'fact')

In [10]:
t = loadText(1)

In [11]:
print(t[:1500])

{Author, Unknown} 

{Author, Unknown} СМИ Ирана: Финляндия не хочет вступать в НАТО из страха перед Россией?

{Author, Unknown} inosmi.ru

{Author, Unknown} 5-го, 2016


{Author, Unknown} Зачем американцам понадобилось перебросить 250 своих военных на северо-восток Сирии, к окрестностям города Румийлан, разбирался обозреватель Khorasan (
{Author, Unknown} 30.04) Алиреза Резахах (Alireza Rezakhah). 
{Author, Unknown} Согласно заявлениям из Пентагона, цель США — способствовать освобождению от боевиков ИГИЛ стратегически важного города Ракка. 
{Author, Unknown} Однако в Дамаске действия Вашингтона резко осудили и назвали покушением на территориальный суверенитет Сирии. 
{Author, Unknown} Примерно в аналогичном ключе высказалась и Москва, где «несогласованные с Дамаском» шаги Белого дома назвали фактором, вызывающим озабоченность.


{Author, Unknown} Почему же и в Москве и Дамаске, которые также ведут борьбу прот


u'nomn'

In [14]:
entities[~entities.entity.isin(["Unknown", "Author"])].head(10)

,entity,entity_car,pos_1,pos_2
4,СМИ,ORG,39,42
5,Ирана,LOC,43,48
6,Финляндия,GEOPOLIT,50,59
7,НАТО,ORG,80,84
8,Россией,GEOPOLIT,101,108
11,ru,ORG,137,139
16,Сирии,LOC,271,276
17,Румийлан,LOC,300,308
18,Khorasan,ORG,334,342
21,Алиреза Резахах (Alireza Rezakhah,PER,370,403


In [15]:
def Text_to_dict(number):
    t = loadText(number)
    text_dict = t.split("\n\n")
    text_dict = {i:parag for i, parag in enumerate(text_dict)}
    sentence_dict = {i:{} for i in range(len(text_dict))}

    for key, paragraph in text_dict.iteritems():
        paragraph = paragraph.split("{Author, Unknown}")
        for sent_numbet, sentence in enumerate(paragraph):

            sentence = cleanString(sentence)
            if len(sentence) != 0:
                sentence_dict[key][sent_numbet] = sentence
                #tknzr.tokenize()
    return sentence_dict

In [16]:
# for key, value in t.iteritems():
#     print("paragraph number", key)
#     for another_key, another_value in value.iteritems():
        
#         print("sentence_number", another_key)
#         for word in another_value:
#             print word

In [227]:
text = Text_to_dict(10)
entities = transformAnnotation(10)
answer = transformAnswer(10) 

In [215]:
def getEntityPositions(text, entities):
    """
    This bad boy finds (hopefully) all entities in the text
    Collects their exact locations and returns a neat Dataframe with them
    """
    
    ENTITIES = pd.DataFrame(columns=["entity", "paragraph", "sentence", "loc_start", "loc_end"])
    
    for paragraph in text:
        for sentence in text[paragraph]:
            for entity in list(entities.unique()):
                if " " + entity + " " in " " + text[paragraph][sentence] +  " ":
                    loc_start = text[paragraph][sentence].find(entity)
                    loc_end = loc_start + len(entity)
                    
                    to_append =   {"entity":entity, 
                                   "paragraph":paragraph, 
                                   "sentence":sentence,
                                   "loc_start":loc_start,
                                   "loc_end":loc_end}
                    
                    ENTITIES = ENTITIES.append(to_append, ignore_index = True)
    ENTITIES = ENTITIES.drop_duplicates()
    ENTITIES = ENTITIES.sort_values(by=["paragraph", "sentence", "loc_start"])
    return ENTITIES

In [216]:
ENTITIES = getEntityPositions(text, entities)

In [217]:
answer

,entity_1,entity_2,attitude,time
0,Россия,Австралия,neg,current
1,Австралия,Россия,neg,current
2,Обама,ИГ,neg,current
3,Запад,Россия,neg,current
4,Россия,Запад,neg,current
5,Россия,Сирия,neg,current
6,Франция,Россия,pos,current
7,author,Россия,neg,current
8,Обама,Асад,neg,current
9,author,Обама,neg,current


In [218]:
for entity in entities.unique():
    if entity not in ENTITIES.entity.unique():
        print entity

Москва
Шенген
Запад-Россия
Сирии-России
Лондон
Брюссель
прим. авт
Россию
Варшаве
Вашингтон
Киев
Фигаро
Татьяна Кастуева-Жан
Париж


In [219]:
ENTITIES.head(20)

,entity,paragraph,sentence,loc_start,loc_end
0,Мыкола Сирук,1,1,4,16
1,Россия,2,1,159,165
3,Украины,2,2,40,47
4,ООН,2,2,50,53
5,Юрий Сергеев,2,2,54,66
6,Twitter,2,2,77,84
2,Россия,2,2,154,160
7,РФ,2,2,184,186
8,Россия,3,1,39,45
9,Сирии,3,1,136,141


In [210]:
def distances(loc1, loc2):
    """
    returns distances from one entity to another
    """
    paragraph = ENTITIES.loc[loc2].paragraph - ENTITIES.loc[loc1].paragraph
    sentence = ENTITIES.loc[loc2].sentence - ENTITIES.loc[loc1].sentence
    return paragraph, sentence

Теперь нужно идти по табличке с выделенными сущностями и отбирать их по критериям для пар. Если именительный падеж - скорее всего, это актор, если в предложении всего одна сущность, возможно, это отношение автора к ней, если сущности в разных абзацах - они не связаны, и т. д. Сюда бы хорошо добавить синтакснет, чтобы понимать, чем в предложении является сущность.

In [211]:
def getMorphCase(word):
    p = morph.parse(u"аборт")[0]
    return p.tag.case

In [212]:
print(loadText(10))

{Author, Unknown} Идеальный сценарий для Кремля

{Author, Unknown} By Мыкола Сирук, inosmi.ruПосмотреть оригиналноябрь 17-го, 2015


{Author, Unknown} Уже в первые минуты после парижских терактов в минувшую пятницу в экспертной среде стали раздаваться предположения, что от этой трагедии больше всего выиграет Россия. 
{Author, Unknown} В частности и постоянный представитель Украины в ООН Юрий Сергеев написал в Twitter: «В парижской трагедии «ищите женщину» — cherchez la femme — имя ее — Россия, ожидаем приглашений от РФ присоединиться к борьбе с террором».


{Author, Unknown} И первым тревожным признаком того, что Россия уже диктует свои условия, стало то, что из переговоров по мирному урегулированию ситуации в Сирии, была исключена Австралия. 
{Author, Unknown} В частности как сообщила британская газета The Guardian со ссылкой на источники в правительстве, Канберру исключили из переговоров, которые проходят под эгидой Международной группы поддержки Сирии именно по настоянию России. 
{A

In [220]:
ENTITIES

,entity,paragraph,sentence,loc_start,loc_end
0,Мыкола Сирук,1,1,4,16
1,Россия,2,1,159,165
3,Украины,2,2,40,47
4,ООН,2,2,50,53
5,Юрий Сергеев,2,2,54,66
6,Twitter,2,2,77,84
2,Россия,2,2,154,160
7,РФ,2,2,184,186
8,Россия,3,1,39,45
9,Сирии,3,1,136,141


In [229]:
ENTITIES.head(10)

,entity,paragraph,sentence,loc_start,loc_end
0,Мыкола Сирук,1,1,4,16
1,Россия,2,1,159,165
3,Украины,2,2,40,47
4,ООН,2,2,50,53
5,Юрий Сергеев,2,2,54,66
6,Twitter,2,2,77,84
2,Россия,2,2,154,160
7,РФ,2,2,184,186
8,Россия,3,1,39,45
9,Сирии,3,1,136,141


In [232]:
distances(0, 1)

(1, 0)

In [235]:
N_ENTITIES = 3 # глобальный параметр, сколько сущностей будем перебирать для текущей
PAIRS = pd.DataFrame(columns=["entity_1", "entity_2"])
ent_1_number = 0
while ent_1_number < len(ENTITIES):
    entity_1 = ENTITIES.entity[ent_1_number]
    
    for ent_2_number in range(N_ENTITIES):
        try:
            entity_2 = ENTITIES.entity[ent_1_number+ent_2_number+1]
            par_dist, sent_dist = distances(ent_1_number, ent_2_number)

            if par_dist == 0: # в одном абзаце
                if sent_dist == 0: # в одном предложении
                    if entity_1 != entity_2: # не равны друг другу
    #                     if (getMorphCase(entity_1) == u'nomn')\
    #                         and (getMorphCase(entity_2) != u'nomn'):
                        PAIRS = PAIRS.append({"entity_1":entity_1,
                                              "entity_2":entity_2}, ignore_index=True)
            
    
        except:
            continue
    ent_1_number += 1

In [236]:
PAIRS

,entity_1,entity_2
0,Мыкола Сирук,Россия
1,Россия,Украины
2,Россия,Юрий Сергеев
3,Украины,Twitter
4,ООН,РФ
5,Юрий Сергеев,Россия
6,Twitter,Сирии
7,РФ,Австралия


In [20]:
from pymystem3 import Mystem

m = Mystem()
def lemmatize(text, mystem=m):
    try:
        return "".join(m.lemmatize(text)).strip()  
    except:
        return " "

In [15]:
for i in range(10):
    print(lemmatize(t[i]))

{Author,
Unknown}
{Author,
Unknown}
россия
и
кратковременный
успех
в
сирия
